This code reproduces the surface type classification for Winter Arctic and Antarctic using the decision tree from the ATLAS theoretical basis document, version 6. The parameters used are the photon rate, the width of fit, the elevation of the sun, and the background rate.

In [1]:
import pandas as pd
adf = pd.read_csv('gt3r_full.csv')
a3df = pd.read_csv('gt3r_atl03.csv')
adf['background_rate'] = a3df['bckgrd_rate']

#columns: ['lats', 'lons', 'heights', 'dt', 'conf', 'stype', 'ssh_flag', 'gauss', 'photon_rate', 'cloud', 
#'backgr_r_200', 'backgr_r_25', 'background_r_norm', 'mss', 'ocean', 'lpe', 'ib', 'width_of_fit','solar_elevation', 'background_rate']

In [2]:
adf.columns

Index(['lats', 'lons', 'heights', 'dt', 'conf', 'stype', 'ssh_flag', 'gauss',
       'photon_rate', 'cloud', 'backgr_r_200', 'backgr_r_25',
       'background_r_norm', 'mss', 'ocean', 'lpe', 'ib', 'width_of_fit',
       'solar_elevation', 'background_rate'],
      dtype='object')

In [32]:
adf['photon_rate']

0        3.402823e+38
1        3.402823e+38
2        3.402823e+38
3        3.402823e+38
4        3.402823e+38
             ...     
88035    3.402823e+38
88036    3.402823e+38
88037    3.402823e+38
88038    3.402823e+38
88039    3.402823e+38
Name: photon_rate, Length: 88040, dtype: float64

In [127]:
#r_surf: surface photon rate
#P1, P2, P3, P4, W1, W2, theta, B1
#w_s: width of fitted gaussian
import numpy as np

# 15.0
P1 = 0.5
P2 = 2.5
P3 = 11.0
P4 = 14.0
W1 = 0.13
W2 = 0.17
B1 = 4.0


#beams 1, 3, 5 are the strong beams, this is beam 6 (weak)
#target: 88040 matches

# surface type for each segment
def assign_surface_type(row, P1, P2, P3, P4, W1, W2, theta_cntl, B1):

    theta_cntl = np.radians(15.0)
    theta_nlb = np.radians(5.0)
    theta_ref = np.radians(20.0)
    #beam gain: Additionally, to accommodate for the differences in returns for each beam,
    # P1, P2, P3, and P4 should be multiplied by relative gain of each beam (beam_gain): i.e.,
    # Pn = Pn*beam_gain()
    #weak beam divide by four
    
    P1 = P1 * 0.25
    P2 = P2 * 0.25
    P3 = P3 * 0.25
    P4 = P4 * 0.25
    #there's something wrong with r_surf
    r_surf = row['photon_rate']
    #r_surf = 
    w_s = row['width_of_fit']
    solar_elev = row['solar_elevation']
    solar_elev = np.radians(solar_elev)
    #print(solar_elev)
    theta_max = np.max([solar_elev, theta_nlb])
    #r_bkg_norm can be calculated from # of shots and 200-Hz background rates, but it should also be provided.
    r_bkg_norm = row['background_r_norm'] # * np.cos(np.pi/2 - theta_ref) / np.cos(np.pi/2 - theta_max)

    #add Th_pr_ratio
    # if r_surf < 1.5:
    #     return 1
    
    if r_surf <= P1:
        return 0
    
    #2: specular lead low w/bkg
    elif P3 <= r_surf <= P4 and w_s <= W1 and r_bkg_norm <= B1 and solar_elev >= theta_cntl:
        return 2
    
    #3: specular lead low
    elif P3 <= r_surf <= P4 and w_s <= W1 and solar_elev < theta_cntl:
        return 3
    
    #4: specular lead high with bkg
    elif r_surf > P4 and w_s <= W1 and r_bkg_norm <= B1 and solar_elev >= theta_cntl:
        return 4
    
    #5 specular lead high 
    elif r_surf > P4 and w_s <= W1 and solar_elev < theta_cntl:
        return 5
    
    #6 dark lead (smooth)
    elif P1 < r_surf <= P2 and w_s <= W1 and r_bkg_norm <= B1 and solar_elev >= theta_cntl:
        return 6
    
    #7 dark lead (smooth)
    elif P1 < r_surf <= P2 and w_s <= W1 and solar_elev < theta_cntl:
        return 7
    #8 dark lead (rough)
    elif P1 < r_surf <= P2 and W1 < w_s <= W2 and r_bkg_norm <= B1 and solar_elev >= theta_cntl:
        return 8
    #9 dark lead (rough)
    elif P1 < r_surf <= P2 and W1 < w_s <= W2 and solar_elev < theta_cntl:
        return 9
    
    #1 all other ice
    else:
        return 1

In [128]:
adf['surface_type'] = adf.apply(lambda row: assign_surface_type(row, P1, P2, P3, P4, W1, W2, theta_cntl, B1), axis=1)

#backgr_r_25, theta 15: 73560 matches
print(adf['surface_type'].value_counts())
print(adf['stype'].value_counts())

surface_type
1    87934
5      102
3        4
Name: count, dtype: int64
stype
1    73591
2     7602
4     4316
6     1823
8      708
Name: count, dtype: int64
